# Subqueries

In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2

conn = psycopg2.connect("dbname=northwind user=postgres password=password host=localhost port=5433")

Subqueries allows you to query data summarized from a nested query
the return set from a nested query may include:
- A single row with a single column
- Multiple rows with a single column
- Multiple rows and columns



### Display all of the orders by employeeID 1

In [2]:
sql = """
SELECT * 
FROM (SELECT * FROM orders WHERE "EmployeeID" = 1 ) o
"""

data = sqlio.read_sql_query(sql, conn)
data.head(10)

,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10258,ERNSH,1,1996-07-17,1996-08-14,1996-07-23,1,140.51,Ernst Handel,Kirchgasse 6,Graz,None,8010,Austria
1,10270,WARTH,1,1996-08-01,1996-08-29,1996-08-02,1,136.54,Wartian Herkku,Torikatu 38,Oulu,None,90110,Finland
2,10275,MAGAA,1,1996-08-07,1996-09-04,1996-08-09,1,26.93,Magazzini Alimentari Riuniti,Via Ludovico il Moro 22,Bergamo,None,24100,Italy
3,10285,QUICK,1,1996-08-20,1996-09-17,1996-08-26,2,76.83,QUICK-Stop,Taucherstraße 10,Cunewalde,None,01307,Germany
4,10292,TRADH,1,1996-08-28,1996-09-25,1996-09-02,2,1.35,Tradiçao Hipermercados,"Av. Inês de Castro, 414",Sao Paulo,SP,05634-030,Brazil
5,10293,TORTU,1,1996-08-29,1996-09-26,1996-09-11,3,21.18,Tortuga Restaurante,Avda. Azteca 123,México D.F.,None,05033,Mexico
6,10304,TORTU,1,1996-09-12,1996-10-10,1996-09-17,2,63.79,Tortuga Restaurante,Avda. Azteca 123,México D.F.,None,05033,Mexico
7,10306,ROMEY,1,1996-09-16,1996-10-14,1996-09-23,3,7.56,Romero y tomillo,"Gran Vía, 1",Madrid,None,28001,Spain
8,10311,DUMON,1,1996-09-20,1996-10-04,1996-09-26,3,24.69,Du monde entier,"67, rue des Cinquante Otages",Nantes,None,44000,France
9,10314,RATTC,1,1996-09-25,1996-10-23,1996-10-04,2,74.16,Rattlesnake Canyon Grocery,2817 Milton Dr.,Albuquerque,NM,87110,USA


### Display  the employee number and hire date of employee with hire date equal to the earliest date.

In [4]:
sql = """
SELECT "EmployeeID", "LastName", "HireDate"
FROM employees
WHERE "HireDate" = (SELECT min("HireDate") FROM employees)
"""

data = sqlio.read_sql_query(sql, conn)
data.head(10)

,EmployeeID,LastName,HireDate
0,3,Leverling,1992-04-01


### Display table whose products are from the dairy products category

In [5]:
sql = """
SELECT * from products
WHERE "CategoryID" IN (SELECT "CategoryID" from categories where "CategoryName" = 'Dairy Products')
"""

data = sqlio.read_sql_query(sql, conn)
data.head(10)

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,11,Queso Cabrales,5,4,1 kg pkg.,21.0,22,30,30,0
1,12,Queso Manchego La Pastora,5,4,10 - 500 g pkgs.,38.0,86,0,0,0
2,31,Gorgonzola Telino,14,4,12 - 100 g pkgs,12.5,0,70,20,0
3,32,Mascarpone Fabioli,14,4,24 - 200 g pkgs.,32.0,9,40,25,0
4,33,Geitost,15,4,500 g,2.5,112,0,20,0
5,59,Raclette Courdavault,28,4,5 kg pkg.,55.0,79,0,0,0
6,60,Camembert Pierrot,28,4,15 - 300 g rounds,34.0,19,0,0,0
7,69,Gudbrandsdalsost,15,4,10 kg pkg.,36.0,26,0,15,0
8,71,Flotemysost,15,4,10 - 500 g pkgs.,21.5,26,0,0,0
9,72,Mozzarella di Giovanni,14,4,24 - 200 g pkgs.,34.8,14,0,0,0


### Display table with products and difference btw most expensive products compared to other remaining products

In [7]:
sql = """
SELECT "ProductName", "UnitPrice", (SELECT max("UnitPrice") from products) - "UnitPrice" as "DifferenceFromMax" 
FROM products

"""

data = sqlio.read_sql_query(sql, conn)
data.head(10)


,ProductName,UnitPrice,DifferenceFromMax
0,Chai,18.00,245.50
1,Chang,19.00,244.50
2,Aniseed Syrup,10.00,253.50
3,Chef Anton's Cajun Seasoning,22.00,241.50
4,Chef Anton's Gumbo Mix,21.35,242.15
5,Grandma's Boysenberry Spread,25.00,238.50
6,Uncle Bob's Organic Dried Pears,30.00,233.50
7,Northwoods Cranberry Sauce,40.00,223.50
8,Mishi Kobe Niku,97.00,166.50
9,Ikura,31.00,232.50
